In [1]:
from fastai.vision.all import *
import timm

all_models = timm.list_models()
timms = dir(timm.models)
print(timms)

print(sorted(timm.models.list_models("swin_l*")))

# conv_models = sorted([model for model in timm.list_models() if "conv" in model])
# print(conv_models)


['BasicBlock', 'Beit', 'Bottleneck', 'ByoBlockCfg', 'ByoModelCfg', 'ByobNet', 'Cait', 'ClassAttn', 'CoaT', 'ConVit', 'ConvMixer', 'ConvNeXt', 'CrossVit', 'CspNet', 'DLA', 'DPN', 'DaVit', 'DefaultCfg', 'DenseNet', 'EdgeNeXt', 'EfficientFormer', 'EfficientFormerV2', 'EfficientNet', 'EfficientNetFeatures', 'EfficientVit', 'EfficientVitLarge', 'EfficientVitMsra', 'Eva', 'FastVit', 'FeatureDictNet', 'FeatureGraphNet', 'FeatureHookNet', 'FeatureHooks', 'FeatureInfo', 'FeatureListNet', 'FocalNet', 'GhostNet', 'GlobalContextVit', 'GraphExtractNet', 'Hiera', 'HighPerfGpuNet', 'HighResolutionNet', 'HighResolutionNetFeatures', 'InceptionResnetV2', 'InceptionV3', 'InceptionV4', 'LayerScaleBlock', 'LayerScaleBlockClassAttn', 'Levit', 'MaxxVit', 'MaxxVitCfg', 'MaxxVitConvCfg', 'MaxxVitTransformerCfg', 'MetaFormer', 'MetaNeXt', 'MixerBlock', 'MlpMixer', 'MobileNetV3', 'MobileNetV3Features', 'MultiScaleVit', 'MultiScaleVitCfg', 'NASNetALarge', 'Nest', 'NextViT', 'NfCfg', 'NormFreeNet', 'PNASNet5Large'

In [2]:
import torch

print(torch.cuda.is_available())


True


In [3]:
import torch


if torch.cuda.is_available():
    print(f"CUDA is available! PyTorch is using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. PyTorch is using CPU.")

CUDA is available! PyTorch is using GPU: NVIDIA GeForce RTX 4090 Laptop GPU


In [4]:

path = untar_data(URLs.PETS)

additional_images = Path('images')

# failed = verify_images(path/'images')
# print(f"Failed images: {failed}")
# failed.map(Path.unlink)

ims = get_image_files(additional_images)
for i, img in enumerate(ims):
    category = img.parent.name
    
    try:
        im = PILImage.create(img)
    except:
        Path.unlink(img)
        print(f"Failed to open {img}")
        continue

    shutil.copy(src=img, dst=path/'images'/f"{category}_{i}.jpg")

# failed = verify_images(path/'images')
# print(f"Failed images: {failed}")
# failed.map(Path.unlink)

dls = ImageDataLoaders.from_name_re(
    path, 
    get_image_files(path/'images'), 
    pat='(.+)_\d+.jpg', 
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    # item_tfms=Resize(460), 
    batch_tfms=aug_transforms(size=224, min_scale=0.75),
    bs=8)


/opt/miniconda3/envs/work/lib/python3.11/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [5]:

# model = models.resnet50
# model = "convnext_large"
#model = models.resnet18
#model = "convnext_xxlarge"
model = "convnext_base"
# model = "swin_large_patch4_window12_384" #failed
# model = "convnext_tiny"
learn = vision_learner(
    dls, 
    model, 
    metrics=accuracy).to_bf16()

print(learn.dls.vocab)

learn.fine_tune(10)
learn.path = Path('.')
learn.export()


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

['Abyssinian', 'Akita', 'Alaskan Malamutes', 'American Bulldog', 'Bengal', 'Birman', 'Bombay', 'Boxers', 'British_Shorthair', 'Bullmastiff', 'Chow Chows', 'Doberman Pinscher', 'Egyptian_Mau', 'German Shepherds', 'Japanese Tosa', 'Labrador Retrievers', 'Maine_Coon', 'Persian', 'Pit Bulls', 'Presa Canario', 'Ragdoll', 'Rottweilers', 'Russian_Blue', 'Siamese', 'Siberian Huskies', 'Sphynx', 'american_bulldog', 'american_pit_bull_terrier', 'basset_hound', 'beagle', 'boxer', 'chihuahua', 'english_cocker_spaniel', 'english_setter', 'german_shorthaired', 'great_pyrenees', 'grey hound', 'havanese', 'japanese spitz', 'japanese_chin', 'keeshond', 'leonberger', 'miniature_pinscher', 'newfoundland', 'pomeranian', 'pug', 'saint_bernard', 'samoyed', 'scottish_terrier', 'shiba_inu', 'staffordshire_bull_terrier', 'wheaten_terrier', 'yorkshire_terrier']


/opt/miniconda3/envs/work/lib/python3.11/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/miniconda3/envs/work/lib/python3.11/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,0.775177,0.518696,0.858544,01:29


epoch,train_loss,valid_loss,accuracy,time


In [ ]:
plt.figure(figsize=(12, 12))
interp = ClassificationInterpretation.from_learner(learn)
# interp.plot_confusion_matrix()
interp.plot_confusion_matrix(figsize=(12, 12), dpi=80)  # Adjust the size and dpi as needed

In [ ]:
# interp.plot_top_losses(50, nrows=25)
interp.plot_top_losses(50, nrows=25, figsize=(15, 50))

In [ ]:
from fastai.vision.widgets import ImageClassifierCleaner

cleaner = ImageClassifierCleaner(learn)
cleaner


In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
learn.fine_tune(200)
learn.path = Path('.')
learn.export()
